In [1]:
# Import dependencies 
import pandas as pd
import json
import requests
import os
from dotenv import load_dotenv
import numpy as np

In [2]:
# Load the wine.csv into a DataFrame.
wine_reviews_df = pd.read_csv(r'C:\Users\fares\Final_project\wine_Extracted_data.csv')
# Display the first 20 headlines 
wine_reviews_df

,Name,Region,Year,Color,Castes,Alcohol_Percentage,Winery,Judge,Judge_Rating,Review_Date,Review_Notes,Wine_Bottle_Label,URL,Price,Price_Interval,Alcohol_Content,cleaned_review,Color_Shades,Flavor,Sensation
0,.Beb 2007,Regional Alentejo,2007.0,Red,"Cabernet Sauvignon, Syrah e Touriga Nacional",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,15.5,12/2008,Bright red color and initial aromas dominated ...,It has an attractive color and an intense arom...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,Low ($5 - $10),High (12% - 14%),bright red color initial aromas dominated spic...,red,"fruity, spicy","delicate, fresh"
1,.Beb 2009,Regional Alentejo,2009.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Susana Esteban,16.0,11/2012,"Reddish color and very aromatic nose, where th...",The careful choice of the best installments of...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,Low ($5 - $10),Very High (14%+),reddish color aromatic nose notes red fruit co...,red,spice,"round, delicate, balanced, fresh, smooth"
2,.Beb 2010,Regional Alentejo,2010.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,16.0,08/2013,It presents a pleasantly concentrated ruby ​​c...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,Low ($5 - $10),Very High (14%+),presents pleasantly concentrated ruby color ce...,"ruby, red","fruity, floral, spicy","round, delicate, balanced"
3,.Beb 2011,Regional Alentejo,2011.0,Yellow,"Arinto, Roupeiro e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,15.5,09/2012,Bright yellow color and delicately marked nose...,We associated a very old wardrobe vineyard wit...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,Low ($5 - $10),High (12% - 14%),bright yellow color delicately marked nose not...,yellow,"fruity, floral, tropical","smooth, balanced, fresh"
4,.Beb 2012,Regional Alentejo,2012.0,Yellow,"Encruzado, Verdelho e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,16.0,07/2014,It has a slightly pale yellow color and a nose...,We associated the structure and complexity of ...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,Low ($5 - $10),High (12% - 14%),slightly pale yellow color nose rich tropical ...,yellow,"vanilla, floral, tropical","elegant, balanced, fresh, smooth, rich"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6342,Zé da Leonor Reserva 2013,Regional Tejo,2013.0,Yellow,"Arinto, Gouveio e Viognier",13.0,"Casa Agrícola Rebelo Lopes, Lda",Filipe Sevinate Pinto,15.5,12/2014,Yellow color and nose rich in floral and citru...,"Castes: Arinto, Gouveio and Viognier. Zé da Le...",https://osvinhos.blogspot.com/2015/11/3816-ze-...,5.75,Low ($5 - $10),High (12% - 14%),yellow color nose rich floral citrus aromas we...,yellow,"fruity, floral, citrus","smooth, rich, fresh"
6343,Zéfyro 2007,Regional Alentejo,2007.0,White,Viognier,13.0,"Herdade da Madeira Velha Agroalimentar, Lda",Luís Bourbon,16.0,04/2009,"It is a modern wine, where the nose is attract...",NaN,https://osvinhos.blogspot.com/2010/06/1336-zef...,8.75,Low ($5 - $10),High (12% - 14%),modern wine nose attractive fruity elegant ali...,White,"fruity, floral","elegant, fresh"
6344,Zéfyro 2013,Regional Alentejo,2013.0,Yellow,Viognier,12.5,"Herdade da Madeira Velha Agroalimentar, Lda",Luís Bourbon,15.0,04/2016,"Yellow color with delicate gold notes, in the ...","(...) Island, which in the bowels of the deep ...",https://osvinhos.blogspot.com/2017/01/4287-zef...,8.75,Low ($5 - $10),High (12% - 14%),yellow color delicate gold notes nose stands r...,yellow,"fruity, floral","delicate, balanced"
6345,Zom Touriga Nacional 2011,DOC Douro,2011.0,Red,Touriga Nacional,14.5,"Barão de Vilar - Vinhos, S. A.",Rui Cunha,17.0,11/2016,Ruby color avioleted and nose rich in floral a...,Zom is located in the heart of the Douro Inter...

In [ ]:
# Define function to categorize wines into price intervals
def categorize_price(min_price):
    if min_price < 10:
        return "Low ($5 - $10)"
    elif 10 <= min_price < 20:
        return "Affordable ($10 - $20)"
    elif 20 <= min_price < 50:
        return "Mid-Range ($20 - $50)"
    elif 50 <= min_price < 100:
        return "Premium ($50 - $100)"
    else:
        return "Luxury ($100+)"

# Apply the function to create a new column
wine_reviews_df["Price_Interval"] = wine_reviews_df["Price"].apply(categorize_price)

In [ ]:
# Define function to categorize wines into alchohol content intervals
def categorize_alcohol_content(alcohol_content):
    if alcohol_content < 10:
        return "Low (0% - 10%)"
    elif 10 <= alcohol_content < 12:
        return "Moderate (10% - 12%)"
    elif 12 <= alcohol_content < 14:
        return "High (12% - 14%)"
    else:
        return "Very High (14%+)"
    
# Apply the function to create a new column
wine_reviews_df["Alcohol_Content"] = wine_reviews_df["Alcohol_Percentage"].apply(categorize_alcohol_content)

In [3]:
# Define flavor categories with associated keywords
flavor_map = {
    "Floral": ["violet", "iris", "rose", "flowers", "blossom"],
    "Fruity": ["peach", "pear", "apple", "berry", "fruit"],
    "Citrus": ["lemon", "lime", "grapefruit", "orange"],
    "Dark-fruit": ["blackberry", "black cherry", "plum", "dark fruit"],
    "Herbal": ["green pepper", "herbaceous", "mint", "grass"],
    "Buttery": ["butter", "creamy", "vanilla", "toasty"],
    "Spicy": ["spice", "pepper", "clove", "cinnamon"],
    "Earthy": ["leather", "mushroom", "truffle", "earth"],
    "Jammy": ["jammy", "strawberry", "raspberry"],
    "Smoky": ["smoke", "bacon", "charcoal"],
    "Nutty": ["nut", "almond", "hazelnut"],
    "Tropical": ["pineapple", "mango", "banana", "tropical"],
    "Sweet": ["honey", "sugar", "sweet"],
    "Bold": ["intense", "bold"],
    "Fresh": ["fresh", "clean", "crisp"]
}


In [4]:
def classify_flavor(Flavor):
    if pd.isna(Flavor):  # Handle missing values
        return "Unknown"
    
    Flavor_lower = Flavor.lower()  # Convert to lowercase
    
    for category, keywords in flavor_map.items():
        if any(word in Flavor_lower for word in keywords):
            return category  # Return the first matching category
    
    return "Other"  # Default if no keywords match


In [5]:
wine_reviews_df["Flavor Class"] = wine_reviews_df["Flavor"].apply(classify_flavor)


In [6]:
wine_reviews_df.head()

,Name,Region,Year,Color,Castes,Alcohol_Percentage,Winery,Judge,Judge_Rating,Review_Date,...,Wine_Bottle_Label,URL,Price,Price_Interval,Alcohol_Content,cleaned_review,Color_Shades,Flavor,Sensation,Flavor Class
0,.Beb 2007,Regional Alentejo,2007.0,Red,"Cabernet Sauvignon, Syrah e Touriga Nacional",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,15.5,12/2008,...,It has an attractive color and an intense arom...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,Low ($5 - $10),High (12% - 14%),bright red color initial aromas dominated spic...,red,"fruity, spicy","delicate, fresh",Fruity
1,.Beb 2009,Regional Alentejo,2009.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Susana Esteban,16.0,11/2012,...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,Low ($5 - $10),Very High (14%+),reddish color aromatic nose notes red fruit co...,red,spice,"round, delicate, balanced, fresh, smooth",Spicy
2,.Beb 2010,Regional Alentejo,2010.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,16.0,08/2013,...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,Low ($5 - $10),Very High (14%+),presents pleasantly concentrated ruby color ce...,"ruby, red","fruity, floral, spicy","round, delicate, balanced",Fruity
3,.Beb 2011,Regional Alentejo,2011.0,Yellow,"Arinto, Roupeiro e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,15.5,09/2012,...,We associated a very old wardrobe vineyard wit...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,Low ($5 - $10),High (12% - 14%),bright yellow color delicately marked nose not...,yellow,"fruity, floral, tropical","smooth, balanced, fresh",Fruity
4,.Beb 2012,Regional Alentejo,2012.0,Yellow,"Encruzado, Verdelho e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,16.0,07/2014,...,We associated the structure and complexity of ...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,Low ($5 - $10),High (12% - 14%),slightly pale yellow color nose rich tropical ...,yellow,"vanilla, floral, tropical","elegant, balanced, fresh, smooth, rich",Buttery


In [7]:
wine_reviews_df['Flavor Class'].value_counts()

Flavor Class
Other       2721
Fruity      2326
Tropical     483
Spicy        392
Buttery      302
Earthy       123
Name: count, dtype: int64

In [8]:
# combine Flavor and Color in one column 
wine_reviews_df["Flavor_color"] = wine_reviews_df["Color"] + " " + wine_reviews_df["Flavor Class"]

In [9]:
wine_reviews_df.head()

,Name,Region,Year,Color,Castes,Alcohol_Percentage,Winery,Judge,Judge_Rating,Review_Date,...,URL,Price,Price_Interval,Alcohol_Content,cleaned_review,Color_Shades,Flavor,Sensation,Flavor Class,Flavor_color
0,.Beb 2007,Regional Alentejo,2007.0,Red,"Cabernet Sauvignon, Syrah e Touriga Nacional",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,15.5,12/2008,...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,Low ($5 - $10),High (12% - 14%),bright red color initial aromas dominated spic...,red,"fruity, spicy","delicate, fresh",Fruity,Red Fruity
1,.Beb 2009,Regional Alentejo,2009.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Susana Esteban,16.0,11/2012,...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,Low ($5 - $10),Very High (14%+),reddish color aromatic nose notes red fruit co...,red,spice,"round, delicate, balanced, fresh, smooth",Spicy,Red Spicy
2,.Beb 2010,Regional Alentejo,2010.0,Red,"Cabernet Sauvignon, Syrah, Alicante Bouschet e...",14.0,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço,16.0,08/2013,...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,Low ($5 - $10),Very High (14%+),presents pleasantly concentrated ruby color ce...,"ruby, red","fruity, floral, spicy","round, delicate, balanced",Fruity,Red Fruity
3,.Beb 2011,Regional Alentejo,2011.0,Yellow,"Arinto, Roupeiro e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,15.5,09/2012,...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,Low ($5 - $10),High (12% - 14%),bright yellow color delicately marked nose not...,yellow,"fruity, floral, tropical","smooth, balanced, fresh",Fruity,Yellow Fruity
4,.Beb 2012,Regional Alentejo,2012.0,Yellow,"Encruzado, Verdelho e Viognier",13.5,Tiago Mateus Cabaço e Cabaço,Tiago Cabaço e Susana Esteban,16.0,07/2014,...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,Low ($5 - $10),High (12% - 14%),slightly pale yellow color nose rich tropical ...,yellow,"vanilla, floral, tropical","elegant, balanced, fresh, smooth, rich",Buttery,Yellow Buttery
